In [ ]:
!pip install beautifulsoup4

In [ ]:
!pip install openai

In [ ]:
import requests

url = 'https://midasnetwork.us/wp-admin/admin-ajax.php'

headers = {
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:133.0) Gecko/20100101 Firefox/133.0'
}

data = {
    'action': 'filter_papers',
    'paged': '1',
    'journal': '',
    'author': '',
    'title': '',
    'startdate': '2024-10-01',
    'enddate': '',
    'displaydefault': '',
}

resp = requests.post(url, data=data, headers=headers)

In [ ]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(resp.text, 'html.parser')
links = []

for article in soup.find_all('article'):
    link = article.find('a')
    links.append(link.get('href'))

In [ ]:
import json

articles = []

for link in links:
    resp = requests.get(link, headers=headers)
    soup = BeautifulSoup(resp.text, 'html.parser')

    data = soup.find_all('p', {'class', 'elementor-heading-title'})
    
    article = {
        'title': data[0].text,
        'abstract': data[1].text,
        'journal': data[2].text,
        'reference': data[3].text
    }

    refs = data[3].find_all('a')
    article['link'] = refs[1].get('href')

    articles.append(article)

In [ ]:
from openai import OpenAI
client = OpenAI()

for article in articles:
    completion = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {
                "role": "user",
                "content": "Write a haiku about recursion in programming."
            }
        ]
    )

    print(completion.choices[0].message)

    break